# Day 17: Trick Shot

## Part one

You finally decode the Elves' message. `HI`, the message says. You continue searching for the sleigh keys.

Ahead of you is what appears to be a large [ocean trench](https://en.wikipedia.org/wiki/Oceanic_trench). Could the keys have fallen into it? You'd better send a probe to investigate.

The probe launcher on your submarine can fire the probe with any [integer](https://en.wikipedia.org/wiki/Integer) velocity in the `x` (forward) and `y` (upward, or downward if negative) directions. For example, an initial `x,y` velocity like `0,10` would fire the probe straight up, while an initial velocity like `10,-1` would fire the probe forward at a slight downward angle.

The probe's `x,y` position starts at `0,0`. Then, it will follow some trajectory by moving in **steps**. On each step, these changes occur in the following order:

- The probe's `x` position increases by its `x` velocity.
- The probe's `y` position increases by its `y` velocity.
- Due to drag, the probe's `x` velocity changes by 1 toward the value `0`; that is, it decreases by `1` if it is greater than `0`, increases by `1` if it is less than `0`, or does not change if it is already `0`.
- Due to gravity, the probe's y velocity decreases by `1`.

For the probe to successfully make it into the trench, the probe must be on some trajectory that causes it to be within a **target area** after any step. The submarine computer has already calculated this target area (your puzzle input). For example:

```
target area: x=20..30, y=-10..-5
```

This target area means that you need to find initial `x,y` velocity values such that after any step, the probe's `x` position is at least `20` and at most `30`, **and** the probe's y position is at least `-10` and at most `-5`.

Given this target area, one initial velocity that causes the probe to be within the target area after any step is `7,2`:

```
.............#....#............
.......#..............#........
...............................
S........................#.....
...............................
...............................
...........................#...
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTT#TT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
```

In this diagram, `S` is the probe's initial position, `0,0`. The `x` coordinate increases to the right, and the y coordinate increases upward. In the bottom right, positions that are within the target area are shown as `T`. After each step (until the target area is reached), the position of the probe is marked with `#`. (The bottom-right `#` is both a position the probe reaches and a position in the target area.)

Another initial velocity that causes the probe to be within the target area after any step is `6,3`:

```
...............#..#............
...........#........#..........
...............................
......#..............#.........
...............................
...............................
S....................#.........
...............................
...............................
...............................
.....................#.........
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................T#TTTTTTTTT
....................TTTTTTTTTTT
```

Another one is `9,0`:

```
S........#.....................
.................#.............
...............................
........................#......
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTT#
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
```

One initial velocity that **doesn't** cause the probe to be within the target area after any step is `17,-4`:

```
S..............................................................
...............................................................
...............................................................
...............................................................
.................#.............................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT..#.............................
....................TTTTTTTTTTT................................
...............................................................
...............................................................
...............................................................
...............................................................
................................................#..............
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
..............................................................#
```

The probe appears to pass through the target area, but is never within it after any step. Instead, it continues down and to the right - only the first few steps are shown.

If you're going to fire a highly scientific probe out of a super cool probe launcher, you might as well do it with **style**. How high can you make the probe go while still reaching the target area?

In the above example, using an initial velocity of `6,9` is the best you can do, causing the probe to reach a maximum y position of **`45`**. (Any higher initial `y` velocity causes the probe to overshoot the target area entirely.)

Find the initial velocity that causes the probe to reach the highest `y` position and still eventually be within the target area after any step. **What is the highest `y` position it reaches on this trajectory?**

In [1]:
import re

EXTRACT_REGEX = '^target area: x=(\-*[0-9]+)\.\.(\-*[0-9]+), y=(\-*[0-9]+)\.\.(\-*[0-9]+)$'

def load_input(filepath='./data/input_test.txt'):
    node_map = []
    m = None
    with open(f'{filepath}', 'r') as input_f:
        input_content = input_f.read()
        m = re.search(EXTRACT_REGEX, input_content)
        
    return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))

square = load_input()
print(f'x1: {square[0]}, x2: {square[1]}, y1: {square[2]}, y2: {square[3]}')

x1: 20, x2: 30, y1: -10, y2: -5


In [2]:
def is_in_square(position, square):
    x, y = position
    x1_square, x2_square, y1_square, y2_square = square
    
    return (x >= x1_square) and (x <= x2_square) and (y >= y1_square) and (y <= y2_square)

def has_missed_square(position, square):
    return (position[0] > square[1]) or (position[1] < square[2])

square = load_input()
is_in_square([25, -11], square)
has_missed_square([1, 1], square)

False

In [3]:
def trick_shot(velocity, square):
    positions = [[0, 0]]
    current_velo_x, current_velo_y = velocity
    
    while (not is_in_square(positions[-1], square)) and (not has_missed_square(positions[-1], square)):
        new_position_x = positions[-1][0] + current_velo_x
        new_position_y = positions[-1][1] + current_velo_y
        positions.append([new_position_x, new_position_y])
        current_velo_y -= 1
        
        if current_velo_x > 0:
            current_velo_x -= 1
            
    return is_in_square(positions[-1], square), positions

square = load_input()
trick_shot([8, 2], square)

(False, [[0, 0], [8, 2], [15, 3], [21, 3], [26, 2], [30, 0], [33, -3]])

In [4]:
def best_trick_shot(max_range, square):
    best_trick_shot = [0, 0]

    for x in range(0, max_range):
        for y in range(0, max_range):
            in_square, positions = trick_shot([x, y], square)

            if in_square and y > best_trick_shot[1]:
                best_trick_shot = [x, y]
                
    return best_trick_shot

square = load_input()
trick_shot(best_trick_shot(100, square), square)

(True,
 [[0, 0],
  [6, 9],
  [11, 17],
  [15, 24],
  [18, 30],
  [20, 35],
  [21, 39],
  [21, 42],
  [21, 44],
  [21, 45],
  [21, 45],
  [21, 44],
  [21, 42],
  [21, 39],
  [21, 35],
  [21, 30],
  [21, 24],
  [21, 17],
  [21, 9],
  [21, 0],
  [21, -10]])

In [5]:
from functools import reduce

square = load_input('./data/input.txt')
_, t_shot = trick_shot(best_trick_shot(500, square), square)
map_y = list(map(lambda x: x[1], t_shot))
max_height = reduce(lambda a, b: a if a > b else b, map_y)
max_height

35511

Your puzzle answer was `35511`.

## Part two

Maybe a fancy trick shot isn't the best idea; after all, you only have one probe, so you had better not miss.

To get the best idea of what your options are for launching the probe, you need to find **every initial velocity** that causes the probe to eventually be within the target area after any step.

In the above example, there are *`112`* different initial velocity values that meet these criteria:

```
23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
8,-2    27,-8   30,-5   24,-7
```

**How many distinct initial velocity values cause the probe to be within the target area after any step?**

In [6]:
def all_shots(min_range, max_range, square):
    all_shots = []

    for x in range(min_range, max_range):
        for y in range(min_range, max_range):
            in_square, positions = trick_shot([x, y], square)

            if in_square:
                all_shots.append([x, y])
                
    return all_shots

square = load_input()
len(all_shots(-100, 100, square))

112

In [7]:
def nb_correct_shots(square):
    nb_correct_shots = 0
    
    y_max = max(abs(square[2]), abs(square[3]))

    for x in range(square[1] + 1):
        for y in range(-y_max, y_max):
            in_square, positions = trick_shot([x, y], square)

            nb_correct_shots += int(in_square)
                
    return nb_correct_shots

square = load_input()
nb_correct_shots(square)

112

In [8]:
square = load_input('./data/input.txt')
nb_correct_shots(square)

3282

Your puzzle answer was `3282`.